In [1]:
from dateparser import parse

In [5]:
import re
import dateparser
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

WEEKDAYS = {
    "monday": 0,
    "tuesday": 1,
    "wednesday": 2,
    "thursday": 3,
    "friday": 4,
    "saturday": 5,
    "sunday": 6,
}

def parse_natural_date(natural_str: str, reference_timezone: str = "UTC") -> str:
    natural_str_lower = natural_str.strip().lower()
    today = datetime.now(ZoneInfo(reference_timezone))

    match = re.match(r"(next|this)\s+(monday|tuesday|wednesday|thursday|friday|saturday|sunday)", natural_str_lower)
    if match:
        when, weekday_str = match.groups()
        weekday = WEEKDAYS[weekday_str]
        current_weekday = today.weekday()
        days_ahead = (weekday - current_weekday + 7) % 7
        if days_ahead == 0 and when == "next":
            days_ahead = 7
        elif days_ahead == 0 and when == "this":
            days_ahead = 0
        elif when == "next":
            days_ahead += 7
        target_date = today + timedelta(days=days_ahead)
        return target_date.date().isoformat()

    if "first day of next month" in natural_str_lower or "next month start" in natural_str_lower:
        first_of_this_month = today.replace(day=1)
        first_of_next_month = (first_of_this_month + timedelta(days=32)).replace(day=1)
        return first_of_next_month.date().isoformat()

    dt = dateparser.parse(
        natural_str,
        languages=['en'],
        settings={
            'TIMEZONE': reference_timezone,
            'RETURN_AS_TIMEZONE_AWARE': True,
            'PREFER_DATES_FROM': 'future',
            'RELATIVE_BASE': today
        }
    )
    if not dt:
        raise ValueError(f"Unable to parse date from '{natural_str}'.")
    return dt.date().isoformat()


In [6]:
test_inputs = [
    "next Monday", "this Friday", "tomorrow", "next month", "June 25",
    "today", "in 3 days", "next year", "this Sunday", "first day of next month"
]

for input_str in test_inputs:
    try:
        result = parse_natural_date(input_str, reference_timezone="Asia/Kolkata")
        print(f"'{input_str}' ➝ {result}")
    except Exception as e:
        print(f"'{input_str}' ➝ Error: {e}")


'next Monday' ➝ 2025-06-30
'this Friday' ➝ 2025-06-20
'tomorrow' ➝ 2025-06-18
'next month' ➝ 2025-07-17
'June 25' ➝ 2025-06-25
'today' ➝ 2025-06-17
'in 3 days' ➝ 2025-06-20
'next year' ➝ 2026-06-17
'this Sunday' ➝ 2025-06-22
'first day of next month' ➝ 2025-07-01


In [9]:
date = parse("wednesday afternoon")

In [10]:
print(date)

None


In [11]:
def smart_parse_datetime(input_str: str, reference_timezone: str = "UTC") -> datetime:
    try:
        if "T" in input_str:
            dt = datetime.fromisoformat(input_str.replace('Z', '+00:00'))
            if dt.tzinfo is None:
                return dt.replace(tzinfo=ZoneInfo(reference_timezone))
            return dt.astimezone(ZoneInfo(reference_timezone))
        date_part = parse_natural_date(input_str, reference_timezone)
        date_obj = datetime.fromisoformat(date_part).date()
        match = re.search(r'\b(\d{1,2}(?::\d{2})?\s*(?:am|pm))\b', input_str, re.IGNORECASE)
        if not match:
            match = re.search(r'\b(\d{1,2}:\d{2})\b', input_str)
        if not match:
            raise ValueError(f"No time found in '{input_str}'")
        time_str = match.group(1)
        for fmt in ('%I:%M%p', '%I%p', '%H:%M'):
            try:
                time_obj = datetime.strptime(time_str.replace(" ", "").upper(), fmt).time()
                break
            except:
                continue
        else:
            raise ValueError(f"Unrecognized time format in '{time_str}'")
        return datetime.combine(date_obj, time_obj).replace(tzinfo=ZoneInfo(reference_timezone))
    except Exception as e:
        raise ValueError(f"Failed to parse datetime from '{input_str}': {e}")


In [12]:
test_inputs = [
    "next Friday at 3pm",
    "an hour before my 5 PM meeting on Friday",
    "sometime late next week",
    "tomorrow at 10:30am",
    "June 20th morning",
    "next month start at 9am",
    "2025-07-01T14:00:00Z"
]

for input_str in test_inputs:
    try:
        result = smart_parse_datetime(input_str, "Asia/Kolkata")
        print(f"Input: {input_str} -> Parsed: {result.isoformat()}")
    except Exception as e:
        print(f"Input: {input_str} -> Error: {e}")


Input: next Friday at 3pm -> Parsed: 2025-06-27T15:00:00+05:30
Input: an hour before my 5 PM meeting on Friday -> Error: Failed to parse datetime from 'an hour before my 5 PM meeting on Friday': Unable to parse date from 'an hour before my 5 PM meeting on Friday'.
Input: sometime late next week -> Error: Failed to parse datetime from 'sometime late next week': Unable to parse date from 'sometime late next week'.
Input: tomorrow at 10:30am -> Parsed: 2025-06-18T10:30:00+05:30
Input: June 20th morning -> Error: Failed to parse datetime from 'June 20th morning': Unable to parse date from 'June 20th morning'.
Input: next month start at 9am -> Parsed: 2025-07-01T09:00:00+05:30
Input: 2025-07-01T14:00:00Z -> Parsed: 2025-07-01T19:30:00+05:30


In [13]:
date = parse("this friday")


In [14]:
print(date)

None


In [15]:
# Import all necessary libraries at the top
import re
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import calendar
import dateparser

# --- Main Function and its Dependencies ---

WEEKDAYS = {
    "monday": 0, "tuesday": 1, "wednesday": 2,
    "thursday": 3, "friday": 4, "saturday": 5, "sunday": 6
}

def parse_natural_date_advanced(
    natural_str: str,
    reference_timezone: str = "UTC",
    relative_base: datetime = None
) -> str:
    """
    Parses a wide variety of natural language date strings into a standard 'YYYY-MM-DD' format with advanced logic.
    Handles relative terms ('tomorrow', 'this vs. next Friday'), specific dates ('June 20th'),
    and complex calendar concepts ('last weekday of the month').
    """
    natural_str_lower = natural_str.strip().lower()
    
    # Use the provided 'relative_base' for testing, or the actual current time for live use.
    if relative_base:
        today = relative_base.astimezone(ZoneInfo(reference_timezone))
    else:
        today = datetime.now(ZoneInfo(reference_timezone))

    # --- 1. Handle simplest cases first ---
    if natural_str_lower == "today":
        return today.date().isoformat()
    if natural_str_lower == "tomorrow":
        return (today + timedelta(days=1)).date().isoformat()

    # --- 2. Advanced Weekday Logic ("this" vs. "next") ---
    weekday_match = re.search(r"(this|next)?\s*(monday|tuesday|wednesday|thursday|friday|saturday|sunday)", natural_str_lower)
    if weekday_match:
        when, weekday_str = weekday_match.groups()
        target_weekday = WEEKDAYS[weekday_str]
        current_weekday = today.weekday()
        
        days_ahead = (target_weekday - current_weekday + 7) % 7
        
        if when == "next":
            # If user explicitly says "next", always go to the following week.
            days_ahead += 7
        elif when == "this" and days_ahead == 0:
            # If it's Monday and user says "this monday", they mean today.
            pass
        elif days_ahead == 0:
            # If it's Monday and user just says "monday", assume they mean next week's.
             days_ahead = 7
             
        target_date = today + timedelta(days=days_ahead)
        return target_date.date().isoformat()

    # --- 3. Handle End-of-Month and Other Complex Logic ---
    if "last day of this month" in natural_str_lower or "end of this month" in natural_str_lower:
        _, num_days = calendar.monthrange(today.year, today.month)
        target_date = today.replace(day=num_days)
        return target_date.date().isoformat()

    if "last weekday of this month" in natural_str_lower:
        _, num_days = calendar.monthrange(today.year, today.month)
        last_day = today.replace(day=num_days)
        
        if last_day.weekday() == 5: # Saturday
            target_date = last_day - timedelta(days=1)
        elif last_day.weekday() == 6: # Sunday
            target_date = last_day - timedelta(days=2)
        else:
            target_date = last_day
        return target_date.date().isoformat()

    if "first day of next month" in natural_str_lower or "next month start" in natural_str_lower:
        first_of_this_month = today.replace(day=1)
        first_of_next_month = (first_of_this_month + timedelta(days=32)).replace(day=1)
        return first_of_next_month.date().isoformat()

    # --- 4. Fallback to the general-purpose dateparser library for all other cases ---
    parsed_dt = dateparser.parse(
        natural_str,
        languages=['en'],
        settings={
            'TIMEZONE': reference_timezone,
            'RETURN_AS_TIMEZONE_AWARE': True,
            'PREFER_DATES_FROM': 'future',
            'RELATIVE_BASE': today
        }
    )

    if not parsed_dt:
        raise ValueError(f"Unable to parse a valid date from the input: '{natural_str}'.")
    
    return parsed_dt.date().isoformat()


# --- Test Suite ---

def run_all_date_tests():
    """
    Runs a comprehensive suite of tests against the advanced date parser.
    """
    print("--- 🧠 Testing the ADVANCED Universal Date Parser 🧠 ---")

    test_timezone = "America/New_York"
    print(f"Using reference timezone: {test_timezone}\n")

    # We will set a fixed "today" to make the tests predictable, regardless of when they are run.
    # Let's pretend today is Thursday, June 20, 2024.
    mock_today = datetime(2024, 6, 20, 10, 0, 0)
    print(f"--- Running tests with a fixed 'today': {mock_today.strftime('%A, %B %d, %Y')} ---\n")

    test_cases = {
        "today": "2024-06-20",
        "tomorrow": "2024-06-21",
        "this friday": "2024-06-21", # The Friday of the current week
        "next friday": "2024-06-28", # The Friday of the following week
        "saturday": "2024-06-22",     # The upcoming Saturday
        "monday": "2024-06-24",       # The upcoming Monday
        "next monday": "2024-07-01",  # The Monday of the week after next
        "last day of this month": "2024-06-30",
        "last weekday of this month": "2024-06-28",
        "first day of next month": "2024-07-01",
        "June 28, 2024": "2024-06-28",
        "in 2 weeks": (mock_today + timedelta(weeks=2)).strftime('%Y-%m-%d')
    }

    all_passed = True
    for text_input, expected_output in test_cases.items():
        print(f"--- Input: '{text_input}' ---")
        try:
            # Pass our mock 'today' into the function using the 'relative_base' argument
            parsed_date = parse_natural_date_advanced(text_input, test_timezone, relative_base=mock_today)
            if parsed_date == expected_output:
                print(f"  ✅ PASSED: Got '{parsed_date}'")
            else:
                print(f"  ❌ FAILED: Got '{parsed_date}', but expected '{expected_output}'")
                all_passed = False
        except ValueError as e:
            print(f"  ❌ FAILED with Error: {e}")
            all_passed = False
        print("-" * (len(text_input) + 20))
    
    print("\n--- 🏁 Test Summary 🏁 ---")
    if all_passed:
        print("🎉🎉🎉 ALL TESTS PASSED SUCCESSFULLY! 🎉🎉🎉")
    else:
        print("🔥🔥🔥 SOME TESTS FAILED. PLEASE REVIEW THE LOGS. 🔥🔥🔥")


# --- Run the Tests ---
if __name__ == '__main__':
    run_all_date_tests()

--- 🧠 Testing the ADVANCED Universal Date Parser 🧠 ---
Using reference timezone: America/New_York

--- Running tests with a fixed 'today': Thursday, June 20, 2024 ---

--- Input: 'today' ---
  ✅ PASSED: Got '2024-06-20'
-------------------------
--- Input: 'tomorrow' ---
  ✅ PASSED: Got '2024-06-21'
----------------------------
--- Input: 'this friday' ---
  ✅ PASSED: Got '2024-06-21'
-------------------------------
--- Input: 'next friday' ---
  ✅ PASSED: Got '2024-06-28'
-------------------------------
--- Input: 'saturday' ---
  ✅ PASSED: Got '2024-06-22'
----------------------------
--- Input: 'monday' ---
  ✅ PASSED: Got '2024-06-24'
--------------------------
--- Input: 'next monday' ---
  ✅ PASSED: Got '2024-07-01'
-------------------------------
--- Input: 'last day of this month' ---
  ✅ PASSED: Got '2024-06-30'
------------------------------------------
--- Input: 'last weekday of this month' ---
  ✅ PASSED: Got '2024-06-28'
----------------------------------------------
--- 

In [17]:
# --- Imports and Setup ---
import os
import re
import calendar
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from typing import Optional
import dateparser
from dotenv import load_dotenv
from langchain_core.tools import tool
from tools.google_auth import get_calendar_service # Assumes google_auth.py is in the same directory or a sibling 'tools' directory

# --- All Required Functions ---
# We include all the necessary functions here to make the script self-contained.

load_dotenv()

GOOGLE_CALENDAR_ID = os.getenv("GOOGLE_CALENDAR_ID")
user_timezones = {}

WEEKDAYS = {
    "monday": 0, "tuesday": 1, "wednesday": 2,
    "thursday": 3, "friday": 4, "saturday": 5, "sunday": 6
}

def parse_natural_date(natural_str: str, reference_timezone: str = "UTC") -> str:
    natural_str_lower = natural_str.strip().lower()
    today = datetime.now(ZoneInfo(reference_timezone))
    if natural_str_lower == "today": return today.date().isoformat()
    if natural_str_lower == "tomorrow": return (today + timedelta(days=1)).date().isoformat()
    weekday_match = re.search(r"(this|next)?\s*(monday|tuesday|wednesday|thursday|friday|saturday|sunday)", natural_str_lower)
    if weekday_match:
        when, weekday_str = weekday_match.groups()
        target_weekday = WEEKDAYS[weekday_str]
        current_weekday = today.weekday()
        days_ahead = (target_weekday - current_weekday + 7) % 7
        if when == "next": days_ahead += 7
        elif when is None and days_ahead == 0: days_ahead = 7
        target_date = today + timedelta(days=days_ahead)
        return target_date.date().isoformat()
    if "first day of next month" in natural_str_lower:
        first_of_this_month = today.replace(day=1)
        first_of_next_month = (first_of_this_month + timedelta(days=32)).replace(day=1)
        return first_of_next_month.date().isoformat()
    parsed_dt = dateparser.parse(natural_str, languages=['en'], settings={'TIMEZONE': reference_timezone, 'RETURN_AS_TIMEZONE_AWARE': True, 'PREFER_DATES_FROM': 'future', 'RELATIVE_BASE': today})
    if not parsed_dt: raise ValueError(f"Unable to parse date: '{natural_str}'.")
    return parsed_dt.date().isoformat()

def _find_unique_event(service, event_name: str, event_date: Optional[str] = None):
    now_utc = datetime.now(ZoneInfo("UTC"))
    if event_date:
        event_date_iso = parse_natural_date(event_date)
        time_min = datetime.fromisoformat(f"{event_date_iso}T00:00:00").astimezone(ZoneInfo("UTC"))
        time_max = datetime.fromisoformat(f"{event_date_iso}T23:59:59").astimezone(ZoneInfo("UTC"))
    else:
        time_min = now_utc
        time_max = now_utc + timedelta(days=365)
    events_result = service.events().list(calendarId=GOOGLE_CALENDAR_ID, timeMin=time_min.isoformat(), timeMax=time_max.isoformat(), q=event_name, singleEvents=True).execute()
    events = events_result.get('items', [])
    if not events: return None, f"Error: No event named '{event_name}' found."
    if len(events) > 1: return None, f"Ambiguity Error: Found multiple events named '{event_name}'."
    return events[0], None

# --- The Tools to be Tested ---

@tool
def create_meeting(title: str, start_time_str: str, duration_minutes: int, event_timezone: str) -> str:
    """
    Creates a new meeting event on the calendar. Use this when the user's intent is to add a new event.
    Arguments:
    - title: The title of the meeting.
    - start_time_str: The desired start time in natural language (e.g., 'tomorrow at 4pm', 'next Tuesday at 10:30am').
    - duration_minutes: The length of the meeting in minutes.
    - event_timezone: The IANA timezone for the event (e.g., 'America/New_York').
    """
    service = get_calendar_service()
    parse_settings = {'TIMEZONE': event_timezone, 'RETURN_AS_TIMEZONE_AWARE': True}
    dt = dateparser.parse(start_time_str, settings=parse_settings)
    end_dt = dt + timedelta(minutes=duration_minutes)
    event = {'summary': title, 'start': {'dateTime': dt.isoformat(), 'timeZone': event_timezone}, 'end': {'dateTime': end_dt.isoformat(), 'timeZone': event_timezone}}
    created_event = service.events().insert(calendarId=GOOGLE_CALENDAR_ID, body=event).execute()
    return f"Success. Meeting '{title}' created."

@tool
def delete_calendar_event(event_name: str, event_date: str) -> str:
    """
    Finds and deletes a specific event from the calendar. Use this when the user wants to remove an event.
    Arguments:
    - event_name: The title of the event to delete (e.g., 'Team Sync').
    - event_date: The date of the event in natural language (e.g., 'today', 'next Tuesday') to help find the correct one.
    """
    service = get_calendar_service()
    event_to_delete, error = _find_unique_event(service, event_name, event_date)
    if error: return error
    service.events().delete(calendarId=GOOGLE_CALENDAR_ID, eventId=event_to_delete['id']).execute()
    return f"Success. Event '{event_name}' deleted."

@tool
def check_availability(user_id: str, search_date: str, start_time: str, end_time: str, duration_minutes: int, timezone: str) -> str:
    """
    Checks for available time slots in a specific window. If the requested window is busy, it AUTOMATICALLY searches for the next available slot within the next 7 days and suggests it. This is the primary tool for finding open times.
    Arguments:
    - user_id: The unique ID for the user.
    - search_date: The date to search, in natural language (e.g., 'next Monday', 'tomorrow').
    - start_time: The start of the search window (e.g., '9:00AM').
    - end_time: The end of the search window (e.g., '5:00PM').
    - duration_minutes: The desired meeting duration in minutes.
    - timezone: The IANA timezone for the search (e.g., 'Asia/Kolkata').
    """
    try:
        search_date_iso = parse_natural_date(search_date, timezone)
        search_start_date = datetime.fromisoformat(search_date_iso)
    except Exception as e: return f"Date parsing error: {e}"
    user_home_zone_str = user_timezones.get(user_id, timezone)
    try:
        search_tz = ZoneInfo(timezone)
        home_tz = ZoneInfo(user_home_zone_str)
        start_dt_naive = datetime.fromisoformat(f"{search_date_iso}T{datetime.strptime(start_time, '%I:%M%p').strftime('%H:%M:%S')}")
        end_dt_naive = datetime.fromisoformat(f"{search_date_iso}T{datetime.strptime(end_time, '%I:%M%p').strftime('%H:%M:%S')}")
        start_dt_aware = start_dt_naive.replace(tzinfo=search_tz)
        end_dt_aware = end_dt_naive.replace(tzinfo=search_tz)
    except Exception as e: return f"Error parsing date/time. Details: {e}"
    start_utc = start_dt_aware.astimezone(ZoneInfo("UTC"))
    end_utc = end_dt_aware.astimezone(ZoneInfo("UTC"))
    service = get_calendar_service()
    events_result = service.events().list(calendarId=GOOGLE_CALENDAR_ID, timeMin=start_utc.isoformat(), timeMax=end_utc.isoformat(), singleEvents=True, orderBy='startTime').execute()
    busy_slots = events_result.get('items', [])
    available_slots = []
    current_time_utc = start_utc
    meeting_duration = timedelta(minutes=duration_minutes)
    for event in busy_slots:
        event_start_utc = datetime.fromisoformat(event['start'].get('dateTime'))
        if current_time_utc + meeting_duration <= event_start_utc: available_slots.append(current_time_utc)
        current_time_utc = max(current_time_utc, datetime.fromisoformat(event['end'].get('dateTime')))
    if current_time_utc + meeting_duration <= end_utc: available_slots.append(current_time_utc)
    if available_slots:
        formatted_results = []
        for slot_utc in available_slots:
            slot_in_search_tz = slot_utc.astimezone(search_tz)
            slot_in_home_tz = slot_utc.astimezone(home_tz)
            formatted_results.append({"utc_iso_format": slot_utc.isoformat(), "target_timezone_format": slot_in_search_tz.strftime('%I:%M %p (%Z)'), "home_timezone_format": slot_in_home_tz.strftime('%I:%M %p your time (%Z)')})
        return f"Available slots found: {formatted_results}"
    conflict_reason = f" The requested time on {search_date} is fully booked"
    if busy_slots:
        conflicting_names = [event.get('summary', 'Untitled') for event in busy_slots]
        conflict_reason += f" by: {', '.join(conflicting_names)}."
    else: conflict_reason += "."
    for i in range(1, 8):
        next_day_to_check = search_start_date + timedelta(days=i)
        window_start = next_day_to_check.replace(hour=9, minute=0, second=0, microsecond=0, tzinfo=search_tz)
        window_end = next_day_to_check.replace(hour=17, minute=0, second=0, microsecond=0, tzinfo=search_tz)
        start_utc_fb, end_utc_fb = window_start.astimezone(ZoneInfo("UTC")), window_end.astimezone(ZoneInfo("UTC"))
        fb_events_result = service.events().list(calendarId=GOOGLE_CALENDAR_ID, timeMin=start_utc_fb.isoformat(), timeMax=end_utc_fb.isoformat(), singleEvents=True, orderBy='startTime').execute()
        fb_busy_slots = fb_events_result.get('items', [])
        fb_current_time = start_utc_fb
        for event in fb_busy_slots:
            event_start_utc = datetime.fromisoformat(event['start'].get('dateTime'))
            if fb_current_time + meeting_duration <= event_start_utc:
                suggestion = f"The next available slot is on {fb_current_time.astimezone(home_tz).strftime('%A, %B %d at %I:%M %p %Z')}."
                return f"{conflict_reason} {suggestion}"
            fb_current_time = max(fb_current_time, datetime.fromisoformat(event['end'].get('dateTime')))
        if fb_current_time + meeting_duration <= end_utc_fb:
            suggestion = f"The next available slot is on {fb_current_time.astimezone(home_tz).strftime('%A, %B %d at %I:%M %p %Z')}."
            return f"{conflict_reason} {suggestion}"
    return f"{conflict_reason} No other availability was found in the next 7 days."

# --- Test Runner ---

def run_availability_test():
    """
    Runs a comprehensive test scenario to validate the intelligent check_availability tool.
    It creates a conflict, tests the tool's ability to resolve it, checks a known-free slot, and cleans up after itself.
    """
    print("--- 📅 Testing the Intelligent Availability Tool 📅 ---")
    
    test_user_id = "notebook_test_user"
    test_timezone = "America/New_York"
    blocker_event_name = "CONFLICT_EVENT_FOR_TESTING"
    blocker_event_date_str = "next Tuesday"
    blocker_event_time_str = f"{blocker_event_date_str} 2:00pm"

    print(f"\n[1. SETUP] 📝 Creating a blocker event: '{blocker_event_name}' at '{blocker_event_time_str}'...")
    try:
        user_timezones[test_user_id] = test_timezone
        creation_result = create_meeting.invoke({
            "title": blocker_event_name,
            "start_time_str": blocker_event_time_str,
            "duration_minutes": 60,
            "event_timezone": test_timezone
        })
        print(f"  > ✅ SETUP COMPLETE: {creation_result}")
    except Exception as e:
        print(f"  > ❌ SETUP FAILED: {e}. Cannot proceed with test. Check your google_auth.py and credentials.json setup.")
        return

    print(f"\n[2. TEST CONFLICT] 💥 Checking availability for the exact time of the blocker event...")
    conflict_result = check_availability.invoke({
        "user_id": test_user_id,
        "search_date": blocker_event_date_str,
        "start_time": "2:00PM",
        "end_time": "3:00PM",
        "duration_minutes": 30,
        "timezone": test_timezone
    })
    print(f"  > 🗣️ Tool Response: {conflict_result}")
    assert "fully booked" in conflict_result and "next available slot" in conflict_result
    print("  > ✅ VERIFIED: Tool correctly identified conflict and suggested an alternative.")

    print(f"\n[3. TEST HAPPY PATH] ☀️ Checking availability for a time that should be open...")
    happy_path_result = check_availability.invoke({
        "user_id": test_user_id,
        "search_date": "next Saturday",
        "start_time": "10:00AM",
        "end_time": "11:00AM",
        "duration_minutes": 30,
        "timezone": test_timezone
    })
    print(f"  > 🗣️ Tool Response: {happy_path_result}")
    assert "Available slots found" in happy_path_result
    print("  > ✅ VERIFIED: Tool correctly found an open slot.")
    
    print(f"\n[4. CLEANUP] 🗑️ Deleting the blocker event...")
    try:
        cleanup_result = delete_calendar_event.invoke({
            "event_name": blocker_event_name,
            "event_date": blocker_event_date_str
        })
        print(f"  > ✅ CLEANUP COMPLETE: {cleanup_result}")
    except Exception as e:
        print(f"  > ❌ CLEANUP FAILED: {e}. Please manually delete the event '{blocker_event_name}' from your calendar.")

    print("\n--- 🎉 Test Scenario Finished Successfully! 🎉 ---")

# --- Execute the Test ---
if __name__ == '__main__':
    run_availability_test()

--- 📅 Testing the Intelligent Availability Tool 📅 ---

[1. SETUP] 📝 Creating a blocker event: 'CONFLICT_EVENT_FOR_TESTING' at 'next Tuesday 2:00pm'...
  > ❌ SETUP FAILED: unsupported operand type(s) for +: 'NoneType' and 'datetime.timedelta'. Cannot proceed with test. Check your google_auth.py and credentials.json setup.


In [1]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

c:\Users\Debajyoti\AppData\Local\Programs\Python\Python312\Lib\site-packages\langsmith\client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [2]:
print(prompt)

input_variables=['agent_scratchpad', 'input'] optional_variables=['chat_history'] input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')]

In [ ]:
from SchedulerAgent.prompt import prompt_template
print(prompt_template)

c:\Users\Debajyoti\AppData\Local\Programs\Python\Python312\Lib\site-packages\langsmith\client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} template='\n🤖 **Your Role:** You are an exceptionally intelligent and proactive AI assistant for managing Google Calendar.\n\n🎯 **Your Mission:** Your job is to understand the user\'s intent from the combined input, collect all required details, handle time ambiguity, resolve conflicts, and use the tools accordingly. Be conversational, precise, and user-friendly.\n\n---\n\n📜 **RULES OF ENGAGEMENT**\n\n🆔 **1. User Identity & Input Format**\n- Every user input you receive will be structured as follows:\n  `"The user\'s ID is \'...\' and their name is \'...\'. The user said: [The user\'s actual message]"`\n- You MUST extract the `user_id` and `user_name` from this input string.\n- Always use the user\'s name in your responses.\n- Always pass the extracted `user_id` when required by a tool.\n\n🌍 **2. Critical Timezone Handling Workflow**\n- You MUST determine the user\'s

In [1]:
from scheduler_voice_agent import compile_agent_workflow
agent_workflow = compile_agent_workflow()

In [2]:
from IPython.display import display,Image

text = agent_workflow.get_graph(xray=True).draw_mermaid()
print(text)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	load_memories(load_memories)
	Scheduler_Agent(Scheduler_Agent)
	Agent_Tools(Agent_Tools)
	__end__([<p>__end__</p>]):::last
	Agent_Tools --> Scheduler_Agent;
	__start__ --> load_memories;
	load_memories --> Scheduler_Agent;
	Scheduler_Agent -. &nbsp;continue&nbsp; .-> Agent_Tools;
	Scheduler_Agent -. &nbsp;end&nbsp; .-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

